In [1]:
import torch
from torch import optim
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from torch.utils.data.dataset import IterableDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
# from audtorch.metrics.functional import *
import shutil
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from higgsboom.FuncUtils.DateTime import TradingDays

import torch.nn.functional as F


#### 定义模型

In [2]:
class MyModel(nn.Module):
    def __init__(
            self,
            input_dim: int =769,
            hidden_dim: int = 15,
            output_dim: int = 2):
        super().__init__()
        self.gru1 = nn.GRU(input_dim, hidden_dim, batch_first=True, num_layers=1)
        self.hidden = nn.BatchNorm1d(hidden_dim)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, x1):
        # (bat_size, seq, features)
        l_x1, _ = self.gru1(x1)
        l_x1 = l_x1[:, -1, :] #(bat_size,timesteps, features)
        hidden = self.hidden(l_x1)
        output = self.linear(hidden)

        return output

In [3]:
class Mydataset(IterableDataset):
    def __init__(self,data1, label) -> None:
            super().__init__()
            self.data1 = data1.astype(np.float32)
            self.label = label.astype(np.float32)
    def __len__(self):
        return len(self.data1)
    def __iter__(self):
        for row in range(len(self.data1)):
            yield self.data1[row], self.label[row]

#### 导入特征

In [4]:
import pickle
import numpy as np

# 定义 pickle 文件名列表
pickle_files = [
    '/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2020_output_1.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2020_output_2.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2020_output_3.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2020_output_4.pkl',
    '/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2021_output_1.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2021_output_2.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2021_output_3.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2021_output_4.pkl',
    '/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2022_output_1.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2022_output_2.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2022_output_3.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2022_output_4.pkl',
    '/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2023_output_1.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2023_output_2.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2023_output_3.pkl','/mnt/weka/home/test/yujichong_share/data/fxs_hushen/2023_output_4.pkl',
]
# 初始化一个空列表来存储所有的数据
data_list = []
pretime='5min'
# 循环遍历 pickle 文件名列表，逐个导入 pickle 文件
for file in pickle_files:
    with open(file, 'rb') as f:
        data = pickle.load(f)
        data = np.nan_to_num(data, nan=0)
        data = data.sum(axis=0)
        data_list.append(data)
        #print(data.shape)
        #print(data_list)
# 使用 np.concatenate 将所有的数据合并
X = np.concatenate(data_list, axis=0)
# 删除第 120 列
X = np.delete(X, 120, axis=1)
# 只保留前 225 列
if pretime == '5min':
    X = X[:,:235,:]
elif pretime == '15min':
    X = X[:,:225,:]
else:
    raise ValueError("Invalid pretime value")
print(X.shape)

(949, 235, 769)


#### 将特征处理为15min

In [5]:
n_sample = 0
len_minute=15
x0 = np.zeros(
    (int(X.shape[0] * (X.shape[1] -len_minute+1)), len_minute, X.shape[2]))
for i in range(0, X.shape[0]):
    #for j in range(0,X.shape[1] -len_minute+1,5):
    for j in range(0,X.shape[1] -len_minute+1):
        x1_one = X[i, j:j + len_minute]
        x0[n_sample, :, :] = x1_one
        n_sample += 1
    e_index = n_sample
x0 = x0[:n_sample, :]
x0.shape

(209729, 15, 769)

#### 导入label

In [6]:
import datetime
import logging
logging.disable(logging.CRITICAL)
from higgsboom.MarketData.DDBSecurityMarketDataUtils import *
#标记预测时间
pretime='5min'
DDBSUtils=DDBSecurityMarketDataUtils(config={"Cluster":"Research",
                      "UserName":"maming",
                      "Password":"passwd123456"})

idx_min_df=DDBSUtils.IndexMinuteDataFrame('000300.SH','20200101','20231205',dataSource="TAQ")
idx_min_df['shan0925'] = idx_min_df.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(1) - 1)
idx_min_df = idx_min_df[idx_min_df['shan0925'].notna()]
idx_min_df=idx_min_df.drop('shan0925', axis=1)

idx_min_df['return'] = idx_min_df.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(15) - 1)
#idx_min_df['return']=idx_min_df['return'].shift(-16)

# #变成5min
idx_min_df['return1'] = idx_min_df.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(5) - 1)
#idx_min_df.iloc[:1000].to_csv('grumoxing.csv')
#idx_min_df
# #对Y矩阵构造
idx_min_df['return'] = idx_min_df.groupby('TRADING_DATE')['return'].shift(-30)
idx_min_df['return1'] = idx_min_df.groupby('TRADING_DATE')['return1'].shift(-20)
#idx_min_df.iloc[:1000].to_csv('grumoxing.csv')
#idx_min_df
idx_min_df = idx_min_df[idx_min_df['TRADING_DATE'] < '2023-12-01']

if pretime == '5min':
    idx_min_df['return_up_down'] = idx_min_df['return1'].apply(lambda x: 1 if x > 0 else 0)
    idx_min_df = idx_min_df[idx_min_df['return1'].notna()]
elif pretime == '15min':
    idx_min_df['return_up_down'] = idx_min_df['return'].apply(lambda x: 1 if x > 0 else 0)
    idx_min_df = idx_min_df[idx_min_df['return'].notna()]
else:
    raise ValueError("Invalid pretime value")
Y = idx_min_df.pivot(index='SYMBOL', columns='TRADING_TIME', values='return_up_down').to_numpy().T
print(Y.shape)

#价量
import datetime
import logging
logging.disable(logging.CRITICAL)
from higgsboom.MarketData.DDBSecurityMarketDataUtils import *
#标记预测时间
pretime='5min'
DDBSUtils=DDBSecurityMarketDataUtils(config={"Cluster":"Research",
                      "UserName":"maming",
                      "Password":"passwd123456"})

df_liangjia=DDBSUtils.IndexMinuteDataFrame('000300.SH','20200101','20231205',dataSource="TAQ")
df_liangjia['shan0925'] = df_liangjia.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(1) - 1)
df_liangjia = df_liangjia[df_liangjia['shan0925'].notna()]
df_liangjia=df_liangjia.drop('shan0925', axis=1)
if pretime == '5min':
    df_liangjia['return'] = df_liangjia.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(5) - 1)
if pretime == '15min':
    df_liangjia['return'] = df_liangjia.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(15) - 1)
#将label处理为15min
df_liangjia['ret'] = df_liangjia.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(1) - 1)
df_liangjia['ret'] = df_liangjia.groupby('TRADING_DATE')['ret'].shift(-1)
if pretime == '5min':
    df_liangjia['return'] = df_liangjia.groupby('TRADING_DATE')['return'].shift(-6)
elif pretime == '15min':
    df_liangjia['return'] = df_liangjia.groupby('TRADING_DATE')['return'].shift(-16)
else:
    raise ValueError("Invalid pretime value")
    
df_liangjia = df_liangjia[df_liangjia['TRADING_DATE'] < '2023-12-01']
df_liangjia = df_liangjia[df_liangjia['return'].notna()]

x_liangjia = df_liangjia.groupby('TRADING_DATE').apply(lambda x: x[['ret', 'AMT']].values)
x_liangjia = x_liangjia.tolist()
x_liangjia = np.array(x_liangjia)
x_liangjia.shape

#结合
n_sample = 0
len_minute=15
x0_liangjia = np.zeros(
    (int(x_liangjia.shape[0] * (x_liangjia.shape[1] -len_minute+1)), len_minute, x_liangjia.shape[2]))
for i in range(0, x_liangjia.shape[0]):
    #for j in range(0,X.shape[1] -len_minute+1,5):
    for j in range(0,x_liangjia.shape[1] -len_minute+1):
        x1_one_liangjia = x_liangjia[i, j:j + len_minute]
        x0_liangjia[n_sample, :, :] = x1_one_liangjia
        n_sample += 1
    e_index = n_sample
x0_liangjia = x0_liangjia[:n_sample, :]
#x0_ret=x0_liangjia[:, :,:1]
#x0_amt=x0_liangjia[:, :,1:]
x0_liangjia.shape
X_hunhe = np.concatenate((x0, x0_liangjia), axis=2)
X_hunhe.shape

/tmp/ipykernel_272723/3551740244.py:11: DeprecationWarning: Call to deprecated method IndexMinuteDataFrame. (This function will be deprecated in version 2.0. Use func index_min_data in higgsboom.data.market.cnsecurity.CNSecurityMarketDataUtils instead.)
  idx_min_df=DDBSUtils.IndexMinuteDataFrame('000300.SH','20200101','20231205',dataSource="TAQ")


(209729, 1)


/tmp/ipykernel_272723/3551740244.py:52: DeprecationWarning: Call to deprecated method IndexMinuteDataFrame. (This function will be deprecated in version 2.0. Use func index_min_data in higgsboom.data.market.cnsecurity.CNSecurityMarketDataUtils instead.)
  df_liangjia=DDBSUtils.IndexMinuteDataFrame('000300.SH','20200101','20231205',dataSource="TAQ")


(209729, 15, 771)

#### 定义函数

In [7]:
#标准化函数
def standardize_data(x_data, med_x1, mad_x1):
    n_sample = 0
    # 遍历所有的股票样本
    for i in range(x_data.shape[0]):
        s_index = n_sample
        x1_one = x_data[i, :,:]
        n_sample += 1
        e_index = n_sample
        for k in range(0, x1_one.shape[0]):
            for s in range(0, x1_one.shape[1]):
                tmpmean = med_x1[k][s]
                tmpstd = mad_x1[k][s]
                x_data[s_index:e_index, k, s] = (x_data[s_index:e_index, k, s] - tmpmean) / (tmpstd + 1e-8)
    return x_data
#test日期函数
from higgsboom.FuncUtils.DateTime import TradingDays
from datetime import datetime
def date_create_train(start_train_date,end_train_date):
    date_list = TradingDays() 
    # 将日期字符串转换为datetime类型
    dates_all_train = [datetime.strptime(i, '%Y-%m-%d') for i in date_list]
    # 筛选出小于'1994-01-27'的日期
    datefilt_train = [day.strftime('%Y%m%d') for day in dates_all_train if datetime.strptime(start_train_date, '%Y-%m-%d') <=day <= datetime.strptime(end_train_date, '%Y-%m-%d')]
    len_train_date=len(datefilt_train)
    return len_train_date
def date_create_test(start_test_date,end_test_date):
    date_list = TradingDays() 
    # 将日期字符串转换为datetime类型
    dates_all = [datetime.strptime(i, '%Y-%m-%d') for i in date_list]
    # 筛选出小于'1994-01-27'的日期
    datefilt = [day.strftime('%Y%m%d') for day in dates_all if datetime.strptime(start_test_date, '%Y-%m-%d') <=day <= datetime.strptime(end_test_date, '%Y-%m-%d')]
    len_test_date=len(datefilt)
    return len_test_date
#start_end日期函数
def date_create_start_end(start_date,end_date):
    date_list = TradingDays() 
    # 将日期字符串转换为datetime类型
    dates_all_start_end = [datetime.strptime(i, '%Y-%m-%d') for i in date_list]
    # 筛选出小于'1994-01-27'的日期
    datefilt_start_end = [day.strftime('%Y%m%d') for day in dates_all_start_end if datetime.strptime(start_date, '%Y-%m-%d') <=day <= datetime.strptime(end_date, '%Y-%m-%d')]
    len_date=len(datefilt_start_end)
    return len_date

#### 模型训练

In [41]:
from datetime import datetime
#加载基本数据
#设置随机数种子
#torch.manual_seed(42)
#torch.manual_seed(50)
torch.manual_seed(65)
# 一些参数
pretime="5min"
if pretime == '5min':
    one_day_sample=221
if pretime == '15min':
    one_day_sample=211
#gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 定义数据导入
def load_data(start_date,start_train_date,end_train_date,start_test_date,end_date):
    
    len_date=date_create_start_end(start_date,end_date)
    print(len_date)
    len_train_date=date_create_train(start_train_date,end_train_date)
    print(len_train_date)
    len_test_date=date_create_test(start_test_date,end_date)
    print(len_test_date)
    
    start_sample=0
    end_sample=len_date*one_day_sample
    
    # 取出对应时间跨度的X和Y
    x1 = X_hunhe[start_sample:end_sample, :, :]
    x1 = np.nan_to_num(x1, nan=0)
    y = Y[start_sample:end_sample]
    y=np.nan_to_num(y, nan=0)

    x1_in_sample = x1[end_sample-(len_test_date+len_train_date)*one_day_sample:end_sample-len_test_date*one_day_sample]
    y_in_sample = y[end_sample-(len_test_date+len_train_date)*one_day_sample:end_sample-len_test_date*one_day_sample]
    print(x1_in_sample.shape)
    print(y_in_sample.shape)
    mask = np.any(x1_in_sample[:,:,:769] != 0, axis=(1, 2))
    x1_in_sample = x1_in_sample[mask]
    y_in_sample = y_in_sample[mask]
    
    x1_test = x1[end_sample-len_test_date*one_day_sample:]
    y_test = y[end_sample-len_test_date*one_day_sample:]
    y_test0=y_test
    mask_test = np.any(x1_test[:,:,:769] != 0, axis=(1, 2))
    x1_test = x1_test[mask_test]
    y_test = y_test[mask_test]

    print(x1_in_sample.shape)
    print(y_in_sample.shape)    

    split1 = int(y_in_sample.shape[0] * 0.9)
    x1_train = x1_in_sample[:split1, :, :]
    x1_val = x1_in_sample[split1:, :, :]
    y_train = y_in_sample[:split1, :]
    y_val = y_in_sample[split1:, :]

    # 对训练集进行标准化
    x1_train_std = x1_train[:,:,-2:]
    print(x1_train_std.shape)
    med_x1_train = np.mean(x1_train_std, 0)
    mad_x1_train = np.std(x1_train_std, 0)
    x1_train_std = standardize_data(x1_train_std, med_x1_train, mad_x1_train)
    #769
    x1_train =x1_train[:,:,:769]
    #769[0]
    #x1_train =x1_train[:,:,685:769]   
    #769+ret+amt
    #x1_train = np.concatenate((x1_train[:,:,:769], x1_train_std), axis=2)
    #ret
    #x1_train=x1_train_std[:,:,:1]
    #amt
    #x1_train=x1_train_std[:,:,-1:]
    
    # 对验证集进行标准化
    x1_val_std = x1_val[:,:,-2:]
    print(x1_val_std.shape)
    med_x1_val = np.mean(x1_val_std, 0)
    mad_x1_val = np.std(x1_val_std, 0)
    x1_val_std = standardize_data(x1_val_std, med_x1_val, mad_x1_val)
    x1_val = np.concatenate((x1_val[:,:,:769], x1_val_std), axis=2)
    
    #769
    x1_val =x1_val[:,:,:769]
    #769[0]
    #x1_val =x1_val[:,:,685:769]    
    #769+ret+amt
    #x1_val = np.concatenate((x1_val[:,:,:769], x1_val_std), axis=2)
    #ret
    #x1_val=x1_val_std[:,:,:1]
    #amt
    #x1_val=x1_val_std[:,:,-1:]
    
    # 对测试集进行标准化
    x1_test_std = x1_test[:,:,-2:]
    print(x1_test_std.shape)
    med_x1_test = np.mean(x1_test_std, 0)
    mad_x1_test = np.std(x1_test_std, 0)
    x1_test_std = standardize_data(x1_test_std, med_x1_test, mad_x1_test)
    x1_test = np.concatenate((x1_test[:,:,:769], x1_test_std), axis=2)

    #769
    x1_test =x1_test[:,:,:769]
    #769[0]
    #x1_test =x1_test[:,:,685:769]
    #769+ret+amt
    #x1_test = np.concatenate((x1_test[:,:,:769], x1_test_std), axis=2)
    #ret
    #x1_test=x1_test_std[:,:,:1]
    #amt
    #x1_test=x1_test_std[:,:,-1:]
    
    print(x1_train.shape)
    print(x1_val.shape)
    print(x1_test.shape)
    print(y_train.shape)
    print(y_val.shape)
    print(y_test.shape)

    return x1_train, x1_val, x1_test, y_train, y_val, y_test,mask_test,y_test0

def CrossEntropyLoss(logits, y_true):
    # x = y_true
    # y = y_pred
    criterion = nn.CrossEntropyLoss()
    loss = criterion(logits, y_true)
    # loss_fn = nn.BCEWithLogitsLoss()
    # loss = loss_fn(y_pred, y_true)
    return loss

BATCH_SIZE = 256
MAX_EPOCH = 200
epoch_iter = 10 #早停轮数设置
w = 0
loss_name = 'CrossEntropy'

# 定义模型训练和测试的方法
def train(train_dl):
    # 模型的训练状态
    acc = 0  # 训练集准确率
    model.train()
    tqdm_ = tqdm(iterable=train_dl)
    
    loss_list = []
    for i, batch in enumerate(tqdm_):
        # 获得一个批次的数据和标签
        x1, labels = batch
        
        x1 = x1.to(device)
        labels = labels.to(device)
        labels = labels.squeeze().long()
        #print(labels.shape)
        #print(labels.shape)
        out = model(x1)
        #print(out.shape)
        # 损失函数
        if loss_name == 'CrossEntropy':
            loss = CrossEntropyLoss(out, labels)
        loss_list.append(loss.item())
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()
        #print(out.shape)
        # print(labels.shape)
        acc += (torch.argmax(out, dim=1) == labels).sum().item()
    # tqdm_.set_description("epoch:{:d} train loss:{:.4f}".format(epoch, loss.item()))
    print(f"train acc:{acc / len(y_train)} train_loss:{sum(loss_list) / len(loss_list)}")
    return acc / len(y_train)

def eval(val_dl):
    # 模型的测试状态
    model.eval()
    acc = 0  # 测试集准确率
    tqdm_ = tqdm(iterable=val_dl)
    for i, batch in enumerate(tqdm_):
        # 获得一个批次的数据和标签
        x1, labels = batch
        x1 = x1.to(device)
        labels = labels.to(device)
        labels = labels.squeeze().long()
        out = model(x1)
        #print(out)
        # 预测正确的数量
        acc+= (torch.argmax(out, dim=1) == labels).sum().item() 
        tqdm_.set_description("epoch:{:d} val {!s}:{:.4f} ".format(epoch, loss_name, acc))
    print(f"val acc:{acc / len(y_val)}")
    return acc / len(y_val)

def test(test_dl):
    # 模型的测试状态
    model_test.eval()
    test_acc=0  # 测试集准确率
    tqdm_test = tqdm(iterable=test_dl)
    result_list=[]
    label_list=[]
    for i, batch in enumerate(tqdm_test):
        # 获得一个批次的数据和标签
        x1_test, labels_test = batch
        x1_test = x1_test.to(device)
        labels_test = labels_test.to(device)
        labels_test = labels_test.squeeze().long()
        y_pred = model(x1_test)
        #print(out)
        # 预测正确的数量
        test_acc+= (torch.argmax(y_pred, dim=1) == labels_test).sum().item() 
        predict_result=F.softmax(y_pred, dim=1)
        result_list.append(predict_result)
        label_list.append(labels_test)
    print(f"test acc:{test_acc / len(y_test)}")
    return test_acc / len(y_test), result_list,label_list


if __name__ == '__main__':
    
    x1_train, x1_val, x1_test,y_train, y_val,y_test,mask_test,y_test0 = load_data('2020-01-01','2021-08-01','2023-07-31','2023-08-01','2023-08-31')
    #x1_train, x1_val ,y_train, y_val = load_data(60)                             
    print('data finish')

    train_ds = Mydataset(x1_train, y_train)
    train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE)

    val_ds = Mydataset(x1_val, y_val)
    val_dl = DataLoader(val_ds, batch_size = BATCH_SIZE)

    test_ds = Mydataset(x1_test, y_test)
    test_dl = DataLoader(test_ds, batch_size = BATCH_SIZE)
    # 定义模型
    model = MyModel()
    model.to(device)
    # 定义优化器
    #optimizer = optim.Adam(model.parameters(), lr=0.0007)# 随机梯度下降
    #optimizer = optim.SGD(model.parameters(), lr=0.0007, momentum=0.9, nesterov=True, weight_decay=1e-4)
    #optimizer = optim.AdamW(model.parameters(), lr=0.0008)
    optimizer = optim.AdamW(model.parameters(), lr=0.0008, weight_decay=0.05)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) # 学习率衰减step_size可设置
    #scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.001) #热重启学习率
    # 记录学习率和 epoch 数组
    lr_history = []
    epoch_history = []

    max_acc = -10000
    max_epoch = 0
    #记录train_ic数组和ic数组
    train_list = []
    val_list = []
    # 定义保存模型参数的列表和计数器
    test_list=[]
    predict_result_list=[]
    final_label_list=[]
    for epoch in range(MAX_EPOCH):
        train_acc = train(train_dl)
        acc = eval(val_dl)

        train_list.append(train_acc)
        val_list.append(acc)

        if acc > max_acc:
            max_acc = acc
            max_epoch = epoch
            model_path = f'12_16滚动输出预测值/model/gru_model/'
            torch.save(model, model_path + 'gru_model' + '.pt')
        else:
            ...
            #if epoch - max_epoch >= epoch_iter:
            #    break
        # 更新学习率
        #scheduler.step()
        
        # 记录学习率和 epoch
        lr_history.append(optimizer.param_groups[0]['lr'])
        epoch_history.append(epoch)

        # #predict
        model_test = MyModel()
        # 加载模型
        model_path = f'12_16滚动输出预测值/model/gru_model/'
        print('--------------------------------')
        print(model_path + 'gru_model' + '.pt')
        print('--------------------------------')
        model_test = torch.load(model_path +  'gru_model' + '.pt')
        test_acc,result_list,label_list=test(test_dl)
        test_list.append(test_acc)
        predict_result_list.append(result_list)
        final_label_list.append(label_list)
    # 创建包含一个子图和共享 x 轴的第二个 y 轴的图形
    fig, ax1 = plt.subplots(figsize=[8, 6])
    ax2 = ax1.twinx()
    # # 绘制学习率随时间的变化折线图
    # ax1.plot(epoch_history, lr_history, label='learning_rate', color='blue')
    # ax1.set_xlabel('Epoch')
    # ax1.set_ylabel('Learning rate')
    # ax1.set_title('Learning rate and scores')
    # ax1.tick_params(axis='y', labelcolor='blue')
    # 绘制验证分数的折线图
    ax1.plot(np.arange(epoch + 1), val_list, label='valid_scores', color='red')
    ax1.set_ylabel('Valid scores')
    ax1.tick_params(axis='y', labelcolor='black')
    # 绘制test分数的折线图
    ax1.plot(np.arange(epoch + 1), test_list, label='test_scores', color='orange')
    ax1.set_ylabel('Test scores')
    ax1.tick_params(axis='y', labelcolor='black')
    # 绘制训练分数的折线图
    ax2.plot(np.arange(epoch + 1), train_list, label='train_scores', color='green')
    ax2.set_ylabel('Train scores')
    ax2.tick_params(axis='y', labelcolor='black')
    # # 绘制验证分数的折线图
    # ax2.plot(np.arange(epoch + 1), val_list, label='valid_scores', color='red')
    # ax2.set_ylabel('Valid scores')
    # ax2.tick_params(axis='y', labelcolor='black')
    # # 绘制test分数的折线图
    # ax2.plot(np.arange(epoch + 1), test_list, label='test_scores', color='orange')
    # ax2.set_ylabel('Test scores')
    # ax2.tick_params(axis='y', labelcolor='black')
    
    # 添加网格线
    ax1.grid(True)
    ax2.grid(True)
    # 显示图形
    plt.show()

890
484
23
(106964, 15, 771)
(106964, 1)
(35794, 15, 771)
(35794, 1)
(32214, 15, 2)
(3580, 15, 2)
(4125, 15, 2)
(32214, 15, 769)
(3580, 15, 769)
(4125, 15, 769)
(32214, 1)
(3580, 1)
(4125, 1)
data finish


100%|██████████| 126/126 [00:00<00:00, 234.02it/s]


train acc:0.5063326504004471 train_loss:0.7043841835998353


epoch:0 val CrossEntropy:1761.0000 : 100%|██████████| 14/14 [00:00<00:00, 294.55it/s]


val acc:0.4918994413407821
--------------------------------
12_16滚动输出预测值/model/gru_model/gru_model.pt
--------------------------------


...

train acc:0.7557273235239337 train_loss:0.4609276333025524


epoch:113 val CrossEntropy:1774.0000 : 100%|██████████| 14/14 [00:00<00:00, 297.79it/s]


val acc:0.49553072625698324
--------------------------------
12_16滚动输出预测值/model/gru_model/gru_model.pt
--------------------------------


 35%|███▌      | 6/17 [00:00<00:00, 205.10it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 23.69 GiB total capacity; 20.83 GiB already allocated; 10.06 MiB free; 22.37 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [42]:
val_max_index = val_list.index(max(val_list))
print(val_max_index)
# 输出第二个列表的该索引的值
print(len(predict_result_list[val_max_index]))

from functools import reduce
#result = [i.tolist() for i in predict_result_list[val_max_index]]
#result = reduce(lambda x, y: x + y, result)
result=np.array(reduce(lambda x, y: x + y, [i.tolist() for i in predict_result_list[val_max_index]]))
print(result)
print(mask_test)
print(len(mask_test))

result1 = np.full((len(mask_test), 2), np.nan)
result1[mask_test] = result

nan_array = np.full((1, 2), np.nan)
for i in range(0,20):
    result1 = np.insert(result1, np.arange(0, len(result1), 221+i), nan_array, axis=0)
print(result1.shape)
result1=np.insert(result1, np.arange(120, len(result1), 241), nan_array, axis=0)
print(result1.shape)

df_result1=pd.DataFrame(result1)
df_result1

import datetime
import logging
logging.disable(logging.CRITICAL)
from higgsboom.MarketData.DDBSecurityMarketDataUtils import *
#标记预测时间
pretime='5min'
DDBSUtils=DDBSecurityMarketDataUtils(config={"Cluster":"Research",
                      "UserName":"maming",
                      "Password":"passwd123456"})

df_time=DDBSUtils.IndexMinuteDataFrame('000300.SH','20230801','20230831',dataSource="TAQ")
df_time=df_time[['TRADING_DATE','TRADING_TIME']]   
df_time = df_time[~df_time['TRADING_TIME'].str.contains('09:25')]
df_time

import pandas as pd

# 将 'TRADING_TIME' 列转换为 datetime 类型
df_time['TRADING_TIME'] = pd.to_datetime(df_time['TRADING_TIME'])

# 定义一个函数，该函数在 11:29 后面添加 11:30
def add_row(group):
    time = pd.Timestamp(group['TRADING_DATE'].iloc[0] + ' 11:30:00')
    if time not in group['TRADING_TIME']:
        new_row = pd.DataFrame({
            'TRADING_DATE': [group['TRADING_DATE'].iloc[0]],
            'TRADING_TIME': [time]
        })
        group = group.append(new_row, ignore_index=True).sort_values('TRADING_TIME')
    return group

# 对每一天的数据应用函数
df_time = df_time.groupby(df_time['TRADING_TIME'].dt.date).apply(add_row).reset_index(drop=True)
df_time

df_final = pd.concat([ df_time,df_result1], axis=1)
df_final

df_final.to_csv('result/hushen300_fxs/769_result/20.csv')

14
17
[[0.67302644 0.32697362]
 [0.68758392 0.31241611]
 [0.68808854 0.31191143]
 ...
 [0.51557553 0.48442447]
 [0.46116361 0.53883636]
 [0.40061685 0.59938312]]
[ True  True  True ...  True  True  True]
5083
(5543, 2)
(5566, 2)


/tmp/ipykernel_272723/449620189.py:37: DeprecationWarning: Call to deprecated method IndexMinuteDataFrame. (This function will be deprecated in version 2.0. Use func index_min_data in higgsboom.data.market.cnsecurity.CNSecurityMarketDataUtils instead.)
  df_time=DDBSUtils.IndexMinuteDataFrame('000300.SH','20230801','20230831',dataSource="TAQ")
/tmp/ipykernel_272723/449620189.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group = group.append(new_row, ignore_index=True).sort_values('TRADING_TIME')


# eval

In [67]:
import pandas as pd
# 创建一个空的DataFrame来存储所有的数据
fxs_pred = pd.DataFrame()

# 遍历1到23
for i in range(1,24):
    # 读取csv文件
    df = pd.read_csv(f'result/hushen300_fxs/769_result/{i}.csv') 
    fxs_pred = pd.concat([fxs_pred, df])

# 重置索引
fxs_pred = fxs_pred.reset_index(drop=True)
#fxs_pred.to_csv('result/hushen300_fxs/769_result/fxs_769_result.csv')

In [68]:
import datetime
import logging
logging.disable(logging.CRITICAL)
from higgsboom.MarketData.DDBSecurityMarketDataUtils import *
#标记预测时间
pretime='5min'
DDBSUtils=DDBSecurityMarketDataUtils(config={"Cluster":"Research",
                      "UserName":"maming",
                      "Password":"passwd123456"})

idx_min_df=DDBSUtils.IndexMinuteDataFrame('000300.SH','20200101','20231205',dataSource="TAQ")
idx_min_df['shan0925'] = idx_min_df.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(1) - 1)
idx_min_df = idx_min_df[idx_min_df['shan0925'].notna()]
idx_min_df=idx_min_df.drop('shan0925', axis=1)

idx_min_df['return'] = idx_min_df.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(15) - 1)
#idx_min_df['return']=idx_min_df['return'].shift(-16)

# #变成5min
idx_min_df['return1'] = idx_min_df.groupby('TRADING_DATE')['CLOSE'].apply(lambda x: x / x.shift(5) - 1)
#idx_min_df.iloc[:1000].to_csv('grumoxing.csv')
#idx_min_df
# #对Y矩阵构造

#idx_min_df.iloc[:1000].to_csv('grumoxing.csv')
#idx_min_df
idx_min_df = idx_min_df[idx_min_df['TRADING_DATE'] < '2023-12-01']

if pretime == '5min':
    idx_min_df['return_up_down'] = idx_min_df['return1'].apply(lambda x: 1 if x > 0 else 0)
    idx_min_df = idx_min_df[idx_min_df['return1'].notna()]
elif pretime == '15min':
    idx_min_df['return_up_down'] = idx_min_df['return'].apply(lambda x: 1 if x > 0 else 0)
    idx_min_df = idx_min_df[idx_min_df['return'].notna()]
else:
    raise ValueError("Invalid pretime value")
idx_min_df=idx_min_df[['TRADING_DATE','TRADING_TIME','return_up_down']]
idx_min_df

/tmp/ipykernel_272723/3425800804.py:11: DeprecationWarning: Call to deprecated method IndexMinuteDataFrame. (This function will be deprecated in version 2.0. Use func index_min_data in higgsboom.data.market.cnsecurity.CNSecurityMarketDataUtils instead.)
  idx_min_df=DDBSUtils.IndexMinuteDataFrame('000300.SH','20200101','20231205',dataSource="TAQ")


TRADING_DATE         TRADING_TIME  return_up_down
6        2020-01-02  2020-01-02 09:35:00               0
7        2020-01-02  2020-01-02 09:36:00               1
8        2020-01-02  2020-01-02 09:37:00               1
9        2020-01-02  2020-01-02 09:38:00               1
10       2020-01-02  2020-01-02 09:39:00               1
...             ...                  ...             ...
229653   2023-11-30  2023-11-30 14:56:00               1
229654   2023-11-30  2023-11-30 14:57:00               1
229655   2023-11-30  2023-11-30 14:58:00               1
229656   2023-11-30  2023-11-30 14:59:00               1
229657   2023-11-30  2023-11-30 15:00:00               0

[223964 rows x 3 columns]

In [69]:
df= pd.merge(fxs_pred, idx_min_df, on=['TRADING_DATE','TRADING_TIME'], how='left')
df

Unnamed: 0 TRADING_DATE         TRADING_TIME   0   1  return_up_down
0                0   2022-01-04  2022-01-04 09:30:00 NaN NaN             NaN
1                1   2022-01-04  2022-01-04 09:31:00 NaN NaN             NaN
2                2   2022-01-04  2022-01-04 09:32:00 NaN NaN             NaN
3                3   2022-01-04  2022-01-04 09:33:00 NaN NaN             NaN
4                4   2022-01-04  2022-01-04 09:34:00 NaN NaN             NaN
...            ...          ...                  ...  ..  ..             ...
112041        5319   2023-11-30  2023-11-30 14:56:00 NaN NaN             1.0
112042        5320   2023-11-30  2023-11-30 14:57:00 NaN NaN             1.0
112043        5321   2023-11-30  2023-11-30 14:58:00 NaN NaN             1.0
112044        5322   2023-11-30  2023-11-30 14:59:00 NaN NaN             1.0
112045        5323   2023-11-30  2023-11-30 15:00:00 NaN NaN             0.0

[112046 rows x 6 columns]

In [70]:
df = df.dropna()
df['pred_y'] = df.apply(lambda row: 0 if row['0'] > row['1'] else 1, axis=1)
df

/tmp/ipykernel_272723/1349273395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pred_y'] = df.apply(lambda row: 0 if row['0'] > row['1'] else 1, axis=1)


Unnamed: 0 TRADING_DATE         TRADING_TIME         0         1  \
20              20   2022-01-04  2022-01-04 09:50:00  0.525821  0.474179   
58              58   2022-01-04  2022-01-04 10:28:00  0.441214  0.558786   
59              59   2022-01-04  2022-01-04 10:29:00  0.478676  0.521324   
60              60   2022-01-04  2022-01-04 10:30:00  0.515337  0.484663   
61              61   2022-01-04  2022-01-04 10:31:00  0.514437  0.485563   
...            ...          ...                  ...       ...       ...   
112024        5302   2023-11-30  2023-11-30 14:39:00  0.849387  0.150613   
112025        5303   2023-11-30  2023-11-30 14:40:00  0.807337  0.192663   
112026        5304   2023-11-30  2023-11-30 14:41:00  0.764303  0.235697   
112027        5305   2023-11-30  2023-11-30 14:42:00  0.723915  0.276085   
112028        5306   2023-11-30  2023-11-30 14:43:00  0.688469  0.311531   

        return_up_down  pred_y  
20                 0.0       0  
58                 1.0       1  
59                 0.0       1  
60                 0.0       0  
61                 0.0       0  
...                ...     ...  
112024             1.0       0  
112025             1.0       0  
112026             0.0       0  
112027             1.0       0  
112028             1.0       0  

[42740 rows x 7 columns]

In [71]:
same_value_percentage = (df['pred_y'] == df['return_up_down']).mean() * 100
same_value_percentage

50.493682732802995

In [72]:
# 假设你的DataFrame是df，日期列的列名是'TRADING_DATE'
# 将日期列转换为日期格式
df['TRADING_DATE'] = pd.to_datetime(df['TRADING_DATE'])

# 设置日期列为索引
df = df.set_index('TRADING_DATE')
df
# 每一个月输出一次same_value_percentage
same_value_percentage_monthly = df.resample('M').apply(lambda x: (x['pred_y'] == x['return_up_down']).mean() * 100)
same_value_percentage_monthly

/tmp/ipykernel_272723/952493558.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TRADING_DATE'] = pd.to_datetime(df['TRADING_DATE'])


TRADING_DATE
2022-01-31    50.564726
2022-02-28    52.729258
2022-03-31    53.238866
2022-04-30    50.331675
2022-05-31    50.097087
2022-06-30    49.618321
2022-07-31    53.436989
2022-08-31    51.135626
2022-09-30    50.957535
2022-10-31    45.489443
2022-11-30    50.352388
2022-12-31    55.545852
2023-01-31    49.121406
2023-02-28    49.189420
2023-03-31    49.381188
2023-04-30    52.006173
2023-05-31    53.454545
2023-06-30    51.500406
2023-07-31    50.201703
2023-08-31    50.448485
2023-09-30    49.065955
2023-10-31    49.799197
2023-11-30    48.036810
Freq: M, dtype: float64